In [1]:
# Imports
# from Code.h5Constructor import h5Constructor

In [2]:
# gc = h5Constructor('./Dataset/CIFs/Train/', './Dataset/h5/Train/')

In [3]:
# gc.gen_h5s(parallelize=False)

In [4]:
from Code.datasetClass import InOrgMatDatasets

test = InOrgMatDatasets('DatasetTest', root='./Dataset/')

Processing...
Done!


In [5]:
test.raw_file_names

['Train/ReO3_CoO3.h5',
 'Train/AntiFlourite_Ir2O.h5',
 'Train/NaCl_FeO.h5',
 'Train/Flourite_MoO2.h5',
 'Train/Rutile_MoO2.h5',
 'Train/NaCl_CoO.h5',
 'Train/Flourite_IrO2.h5',
 'Train/CdI2_IrO2.h5',
 'Train/CdI2_FeO2.h5',
 'Train/ReO3_IrO3.h5',
 'Train/Rutile_CoO2.h5',
 'Train/NaCl_IrO.h5',
 'Train/Spinel_CoCo2O4.h5',
 'Train/ReO3_MoO3.h5',
 'Train/AntiFlourite_Fe2O.h5',
 'Train/Spinel_MoMo2O4.h5',
 'Train/CdI2_CoO2.h5',
 'Train/Spinel_FeFe2O4.h5',
 'Train/AntiFlourite_Co2O.h5',
 'Train/ReO3_FeO3.h5',
 'Train/Flourite_FeO2.h5',
 'Train/AntiFlourite_Mo2O.h5',
 'Train/CdI2_MoO2.h5',
 'Train/Flourite_CoO2.h5',
 'Train/Rutile_FeO2.h5',
 'Train/Spinel_IrIr2O4.h5',
 'Train/Rutile_IrO2.h5',
 'Train/NaCl_MoO.h5']

In [7]:
test.get(100)

Data(
  x=[12, 4],
  edge_index=[2, 64],
  edge_attr=[88],
  y={
    np_size=11.35,
    nd=[580],
    xrd=[580],
    nPDF=[301],
    xPDF=[301],
    sans=[300],
    saxs=[300]
  }
)